In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
def extract_page(pesquisa,page=1):
    p = pesquisa.split()
    p1 = '-'.join(p)
    p2 = '%20'.join(p)
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}
    url = f'https://www.catho.com.br/vagas/{p1}/?q={p2}&page={page}'
    session = requests.get(url,headers)
    soup = BeautifulSoup(session.content,'html.parser')
  
    
    return soup

def transform_page(soup):
    data = {'title':[],'descrição':[],'local':[],'salario':[],'link':[]}
    
    articles = soup.find_all('article')
    for art in articles[3::]:
        try:    data['title'].append(art.h2.text)
        except: data['title'].append('')
            
        try:    data['descrição'].append(art.find('span',{'class':"job-description"}).text)
        except: data['descrição'].append('')
            
        try:    data['local'].append(art.find('button',{'class':"sc-ksYbfQ ehLxPn"}).text)
        except: data['local'].append('')
            
        try:    data['salario'].append(str(art.find('div',{'class':"sc-hqyNC exuLzT"}).text.replace('R$','')))
        except: data['salario'].append('')
            
        try:    data['link'].append(art.a.attrs['href'])
        except: data['link'].append('')
            
    return pd.DataFrame(data)

def get_descrição_completa(links):
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}
    desc_completa = []
    
    for link in links:
        session = requests.get(link,headers)
        soup = BeautifulSoup(session.content,'html.parser')
        
        try:    
            desc_string = soup.find('span',{'class':'job-description'}).text
            desc_completa.append(desc_string.replace('\n','').replace('  ',''))
                
        except: 
            desc_completa.append('')

    return  desc_completa  

In [3]:
%%time
areas = ['Analista de dados','Analista de BI','Analista de controladoria','Analista de FP&A','Planejamento financeiro']

soup = extract_page(areas[0])
df_all = transform_page(soup)

for area in areas[1::]:
    for i in range(1,70):
        soup = extract_page(area,i)
        df = transform_page(soup)
        df_all = pd.concat([df_all,df],ignore_index=True)



Wall time: 9min 35s


In [4]:
df_all = df_all.drop_duplicates(subset='descrição',ignore_index=True)
df_all

,title,descrição,local,salario,link
0,Analista de dados,Será responsável pela construção de relatório...,,,https://www.catho.com.br/vagas/analista-de-dad...
1,Analista de Dados,"Realizar a administração, operação, verificaç...",,,https://www.catho.com.br/vagas/analista-de-dad...
2,ANALISTA DE DADOS,- Compilar e processar dados em grande escala...,,,https://www.catho.com.br/vagas/analista-de-dad...
3,Analista de Dados,"Interpretar dados, analisar resultados usando...",,,https://www.catho.com.br/vagas/analista-de-dad...
4,Analista de Dados,"Administrar banco de dados, envolvendo manute...",,,https://www.catho.com.br/vagas/analista-de-dad...
...,...,...,...,...,...
2364,Gerente Administrativo,"Será responsável em Gerencia, planeja e contr...",,,https://www.catho.com.br/vagas/gerente-adminis...
2365,Analista Administrativo,rogramação de pagamentos e lançamentos no sis...,,,https://www.catho.com.br/vagas/analista-admini...
2366,Assistente Financeiro,Recebimentos de boletos e contas á pagar. Sal...,,,https://www.catho.com.br/vagas/assistente-fina...
2367,Analista Contábil JR,Reportando-se ao Gerente de Contabilidade faz...,,,https://www.catho.com.br/vagas/analista-contab...


In [5]:
links = df_all['link']

In [6]:
%%time
all_description = get_descrição_completa(links)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))

In [7]:
df_all['descrição completa'] = all_description
df_all = df_all.dropna()

NameError: name 'all_description' is not defined

In [ ]:
df_all.to_excel('Cathoyy.xlsx')